# Simulator Wrapper

In [1]:
!pip install pyNetLogo
!pip install JPype1
!pip install GPy==1.10.0
!pip install GPyOpt==1.2.1
!pip install emukit

In [1]:
import numpy as np

Here you should specify the dorectory where you install the NetLogo. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home=r"D:\Programs\NetLogo 6.2.0", netlogo_version="6.2")

Here you should specify the path of the Netlogo model you have downloaded in order to load the model.

In [3]:
netlogo.load_model(r'C:\Users\Chengzu\Downloads\Blood_Sugar_Regulation.nlogo')

In [4]:
problem = {
        'num_vars': 4,
        'names': [
                  'metabolic-rate',
                  'insulin-sensitivity',
                  'glucose-sensitivity',
                  'glucagon-sensitivity',
                  ],
        'bounds': [[0, 200],
                   [0., 1.],
                   [0., 1.],
                   [0., 1.]]
    }

In [5]:
def run_simulation(experiment, start_time_step=90, end_time_step=100):
    """ run a netlogo model
    Parameters
    ----------
    experiments : dict
    """

    # Set the input parameters
    
    for key, value in experiment.items():
        if key == 'random-seed':
            # The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))

    netlogo.command('setup')
    # Run for 100 ticks and return the number of glucose
    counts = netlogo.repeat_report(['count glucoses', 'count insulins', 'count glucagons'], 101)

    results = pd.Series([counts['count glucoses'].values[start_time_step: end_time_step].mean()],
                        index=['Avg. glucoses'])
    return results

# Load Specified Parameters

In [6]:
def specify_parameters(file: str, variable_X: list):
    '''
    The file should be a .csv file, with who, metabolic-rate, insulin-sensitivity, glucose-sensitivity, glucagon-sensitivity as its title.
    The returned params are a list full of lists
    '''
    params = []
    import pandas as pd
    df = pd.read_csv(file)
    for i in range(len(df)):
        item = df.iloc[i].to_dict()
        _ = item.pop("who")
        item_values = []
        for k, v in item.items():
            if k not in variable_X:
                continue
            else:
                if k == "metabolic-rate":
                    v = v/200
                item_values.append(v)
        params.append(item_values)
    return params

# Initialize Simulator

In [7]:
class Simulator:
    def __init__(self, variable_X):
        self.variable_X = variable_X
        
    def simulate(self, input_values):
        assert len(input_values[0]) == len(self.variable_X)
        item = {}
        for k, v in zip(self.variable_X, input_values[0]):
            if k == "metabolic-rate":
                v = v * 200
            item[k] = v
        result = run_simulation(item)
        return np.array([result['Avg. glucoses']]).reshape(-1, 1)
    
    def opt_simulate(self, input_values):
        assert len(input_values[0]) == len(self.variable_X)
        item = {}
        for k, v in zip(self.variable_X, input_values[0]):
            if k == "metabolic-rate":
                v = v * 200
            item[k] = v
        result = run_simulation(item)
        return np.array([np.abs(result['Avg. glucoses'] - 4000)]).reshape(-1, 1)

# Experiments

In [ ]:
filepath = "../my_sample_data.csv"

In [18]:
variable_X = ["metabolic-rate", "insulin-sensitivity", "glucose-sensitivity", "glucagon-sensitivity"]
simulator = Simulator(variable_X)

In [19]:
X = specify_parameters(
    file=filepath,
    variable_X=variable_X
)

In [20]:
Y = [simulator.simulate([p]).reshape(-1)[0] for p in X]